In [161]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE 
import cv2
from glob import glob
import matplotlib.pyplot as plt

image_path = os.path.join("lung/data/Lung Segmentation/CXR_png")
mask_path = os.path.join("lung/data/Lung Segmentation/masks")

In [158]:
check = [i for i in mask if "_mask" in i]
print("Total mask that has modified name:",len(check))

Total mask that has modified name: 704


In [162]:
def getData(X_shape):
    im_array = []
    mask_array = []
    for i in tqdm(images):  
        # Görüntüyü oku
        image_file = os.path.join(image_path, i)
        if not os.path.exists(image_file):
            #print(f"Warning: Image file not found: {image_file}")
            continue
        im = cv2.resize(cv2.imread(image_file), (X_shape, X_shape))[:, :, 0]
        
        # Maskeyi oku
        mask_file = os.path.join(mask_path, i.split(".png")[0] + "_mask.png")
        if not os.path.exists(mask_file):
            #print(f"Warning: Mask file not found: {mask_file}")
            continue
        mask = cv2.resize(cv2.imread(mask_file), (X_shape, X_shape))[:, :, 0]
        
        im_array.append(im)
        mask_array.append(mask)
    
    print("Test images length:", len(im_array))
    print("Test masks length:", len(mask_array))
    
    return im_array, mask_array

im_array, mask_array = getData(256)

100%|█████████████████████████████████████████| 800/800 [01:25<00:00,  9.39it/s]

Test images length: 704
Test masks length: 704


In [163]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(im_array, mask_array, test_size = 0.2, random_state= 42)



In [164]:
X_train = np.array(X_train).reshape(len(X_train),dim,dim,1)
y_train = np.array(y_train).reshape(len(y_train),dim,dim,1)
X_test = np.array(X_test).reshape(len(X_test),dim,dim,1)
y_test = np.array(y_test).reshape(len(y_test),dim,dim,1)
assert X_train.shape == y_train.shape
assert X_test.shape == y_test.shape
images = np.concatenate((X_train,X_test),axis=0)
mask  = np.concatenate((y_train,y_test),axis=0)

In [165]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler


def unet(input_size=(256,256,1)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

In [166]:
from tensorflow.keras.losses import binary_crossentropy

model = unet(input_size=(256,256,1))

model.compile(optimizer=Adam(), loss='binary_crossentropy', 
              metrics=['accuracy'])

#model.summary()

In [167]:
from IPython.display import clear_output
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split

train_vol, validation_vol, train_seg, validation_seg = train_test_split(images/255.0,(mask>127).astype(np.float32), 
                                                                        test_size = 0.1,random_state = 127)

train_vol, test_vol, train_seg, test_seg = train_test_split(train_vol,train_seg,
test_size = 0.1,
random_state = 127)

loss_history = model.fit(x = train_vol,
                         y = train_seg,
                         batch_size = 8,
                  epochs = 5)

Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.7439 - loss: 0.5555
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.8267 - loss: 0.3714
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.9217 - loss: 0.2040
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9428 - loss: 0.1542
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9522 - loss: 0.1305
